In [71]:
import numpy as np
import pandas as pd 
from matplotlib import pyplot as plt

In [72]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [76]:
train.head()

,portfolio_id,desk_id,office_id,pf_category,start_date,sold,country_code,euribor_rate,currency,libor_rate,...,indicator_code,sell_date,type,hedge_value,status,return,country_code_enc,currency_enc,pf_cat_enc,portfolio_id_enc
0,PF00001002,DSK00001001,OFF00001002,B,20040720,110000000.0,T,0.02074,USD,2.332216,...,NaN,20040812,B,NaN,NaN,0.02496,2,4,1,1
1,PF00001003,DSK00001002,OFF00001001,A,20040709,176671000.0,N,0.02074,GBP,5.269617,...,NaN,20040812,C,NaN,NaN,0.05496,1,2,0,2
2,PF00001005,DSK00001004,OFF00001001,A,20040723,56474000.0,T,0.02074,USD,2.332216,...,NaN,20040817,A,NaN,NaN,0.02496,2,4,0,4
3,PF00001006,DSK00001005,OFF00001001,A,20040609,164813000.0,T,0.02074,USD,2.332216,...,NaN,20040713,A,NaN,NaN,0.02496,2,4,0,5
4,PF00001007,DSK00001005,OFF00001002,B,20040609,140800000.0,T,0.02074,USD,2.332216,...,NaN,20040713,B,NaN,NaN,0.02496,2,4,1,6


In [74]:
from sklearn.preprocessing import LabelEncoder

In [86]:
label_encoder = LabelEncoder()

#create the encoding for loop chutiye
label_encoder.fit(np.hstack([train.country_code, test.country_code]))
train['country_code_enc'] = pd.DataFrame(label_encoder.transform(train.country_code))
test['country_code_enc'] = pd.DataFrame(label_encoder.transform(test.country_code))

label_encoder.fit(np.hstack([train.currency, test.currency]))
train['currency_enc'] = pd.DataFrame(label_encoder.transform(train.currency))
test['currency_enc'] = pd.DataFrame(label_encoder.transform(test.currency))

label_encoder.fit(np.hstack([train.pf_category, test.pf_category]))
train['pf_cat_enc'] = pd.DataFrame(label_encoder.transform(train.pf_category))
test['pf_cat_enc'] = pd.DataFrame(label_encoder.transform(test.pf_category))

label_encoder.fit(np.hstack([train.portfolio_id, test.portfolio_id]))
train['portfolio_id_enc'] = pd.DataFrame(label_encoder.transform(train.portfolio_id))
test['portfolio_id_enc'] = pd.DataFrame(label_encoder.transform(test.portfolio_id))

train['desk_id'].fillna("DSK00001000", inplace=True)
test['desk_id'].fillna("DSK00001000", inplace=True)
label_encoder.fit(np.hstack([train.desk_id, test.desk_id]))
train['desk_id_enc'] = pd.DataFrame(label_encoder.transform(train.desk_id))
test['desk_id_enc'] = pd.DataFrame(label_encoder.transform(test.desk_id))

label_encoder.fit(np.hstack([train.office_id, test.office_id]))
train['office_id_enc'] = pd.DataFrame(label_encoder.transform(train.office_id))
test['office_id_enc'] = pd.DataFrame(label_encoder.transform(test.office_id))


In [134]:
features = train.columns
features

Index(['euribor_rate', 'libor_rate', 'return', 'country_code_enc',
       'currency_enc', 'pf_cat_enc', 'portfolio_id_enc', 'office_id_enc',
       'desk_id_enc', 'balance', 'ind_code_enc', 'type_enc', 'hedge_value_enc',
       'status_enc'],
      dtype='object')

In [132]:
test = test.drop(['portfolio_id','desk_id','office_id','pf_category','start_date','sold','country_code','currency','bought','creation_date','indicator_code','sell_date','type','hedge_value','status'],axis=1)

In [138]:
train = train.drop(['return'],axis=1)

In [91]:
train['balance'] = train.sold - train.bought
test['balance'] = test.sold - test.bought

In [126]:
imp_feat = ['ind_code_enc','type_enc','hedge_value_enc','status_enc','euribor_rate','libor_rate','country_code_enc','currency_enc','pf_cat_enc','portfolio_id_enc','office_id_enc','desk_id_enc','balance']
imp_feat

['ind_code_enc',
 'type_enc',
 'hedge_value_enc',
 'status_enc',
 'euribor_rate',
 'libor_rate',
 'country_code_enc',
 'currency_enc',
 'pf_cat_enc',
 'portfolio_id_enc',
 'office_id_enc',
 'desk_id_enc',
 'balance']

In [165]:
train.desk_id_enc.isnull().value_counts()
#train['libor_rate'].fillna(0, inplace=True)

False    9366
Name: desk_id_enc, dtype: int64

In [166]:
ps1_train = train.copy()


In [110]:
train['indicator_code'].fillna(False, inplace=True)
test['indicator_code'].fillna(False, inplace=True)
label_encoder.fit(np.hstack([train.indicator_code, test.indicator_code]))
train['ind_code_enc'] = pd.DataFrame(label_encoder.transform(train.indicator_code))
test['ind_code_enc'] = pd.DataFrame(label_encoder.transform(test.indicator_code))

label_encoder.fit(np.hstack([train.type, test.type]))
train['type_enc'] = pd.DataFrame(label_encoder.transform(train.type))
test['type_enc'] = pd.DataFrame(label_encoder.transform(test.type))

train['hedge_value'].fillna(False, inplace=True)
test['hedge_value'].fillna(False, inplace=True)
label_encoder.fit(np.hstack([train.hedge_value, test.hedge_value]))
train['hedge_value_enc'] = pd.DataFrame(label_encoder.transform(train.hedge_value))
test['hedge_value_enc'] = pd.DataFrame(label_encoder.transform(test.hedge_value))

train['status'].fillna(False, inplace=True)
test['status'].fillna(False, inplace=True)
label_encoder.fit(np.hstack([train.status, test.status]))
train['status_enc'] = pd.DataFrame(label_encoder.transform(train.status))
test['status_enc'] = pd.DataFrame(label_encoder.transform(test.status))


In [130]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(random_state = 0)

In [172]:
rf.fit(ps1_train,y)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=0, verbose=0, warm_start=False)

In [180]:
pred = rf.predict(ps1_test)
pred

array([ 0.02525478,  0.02525478,  0.02525478, ...,  0.00614833,
        0.00547173,  0.00590644])

In [173]:
ps1_test = test.copy()

In [178]:
ps1_test.euribor_rate.isnull().value_counts()
#ps1_test.fillna(0, inplace=True)

False    4801
Name: euribor_rate, dtype: int64

In [181]:
sub = pd.DataFrame({'return':pred})

In [182]:
sub.to_csv('pred.csv')